In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
raw = pd.read_csv('./data/raw/sales_train_validation.csv')
sampsub = pd.read_csv('./data/raw/sample_submission.csv')
df = raw.copy()

In [7]:
def quantile_model(item_data):
    quantiles_to_predict = [0.005, 0.025, 0.165, 0.25, 0.5, 1-0.25, 1-0.165, 1-0.025, 1-0.005]
    quantiles = item_data['value'].quantile(quantiles_to_predict)
    return(quantiles)

In [8]:
def build_item_quantile_model(item_id, store_id):
    identifier = '{}_{}_validation'.format(item_id, store_id)
    item_data = df[df['id'] == identifier].melt(id_vars = ['id', 'item_id', 'dept_id', \
                                                           'cat_id', 'store_id','state_id',]).drop('variable', axis = 1)
    #model
    quantiles = quantile_model(item_data)
    
    quantiles_to_predict = [0.005, 0.025, 0.165, 0.25, 0.5, 1-0.25, 1-0.165, 1-0.025, 1-0.005]
    for q in quantiles_to_predict:
        sub_row_id = '{}_{}_{:.3f}_{}'.format(item_id, store_id, q, 'validation')
        model_dict[sub_row_id] = {'id': identifier, 'quantile': "{:.3f}".format(q), 'set': 'validation', 'prediction': quantiles[q]}
        sub_row_id = '{}_{}_{:.3f}_{}'.format(item_id, store_id, q, 'evaluation')
        model_dict[sub_row_id] = {'id': identifier, 'quantile': "{:.3f}".format(q), 'set': 'evaluation', 'prediction': quantiles[q]}
        

In [9]:
def build_agg_models(item_predictions, aggregations):
    agg_dict = {}
    for agg, info in aggregations.items():
        prefix = 'Total' if agg == 'total' else ''
        agg_df = pd.DataFrame(item_predictions.groupby(info['group'] + ['quantile', 'set'])['prediction'].sum()).reset_index()
        agg_df['sub_id'] = agg_df.apply(lambda x: prefix + '_'.join([x[c] for c in info['group']]) + info['sep'] + x['quantile'] + '_' + x['set'], axis = 1)
        agg_dict[agg] = agg_df
    return(agg_dict)

In [10]:
model_dict = {}
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    build_item_quantile_model(item_id = row['item_id'], store_id = row['store_id'])

100%|██████████| 30490/30490 [05:55<00:00, 85.88it/s]


In [11]:
item_predictions = pd.DataFrame(model_dict).transpose().reset_index().merge(df, left_on = 'id', right_on = 'id')

In [12]:
aggregations = {
    'total': {'group': [], 'sep': '_X_'}, 
    'state': {'group':['state_id'], 'sep': '_X_'},
    'stores': {'group': ['store_id'], 'sep': '_X_'}, 
    'category': {'group':['cat_id'], 'sep':'_X_'},
    'state_cat':{'group': ['state_id', 'cat_id'], 'sep':'_'},
    'state_dept': {'group':['state_id', 'dept_id'], 'sep': '_'},
    'state_item': {'group':['state_id', 'item_id'], 'sep':'_'},
    'store_cat': {'group': ['store_id', 'cat_id'], 'sep':'_'},
    'store_dept': {'group': ['store_id', 'dept_id'], 'sep':'_'},
    'dept': {'group': ['dept_id'], 'sep':'_X_'},
    'item':{'group':['item_id'], 'sep':'_X_'}, 
    
}

In [13]:
agg_dict = build_agg_models(item_predictions, aggregations)

In [14]:
all_agg = pd.DataFrame(columns = ['sub_id', 'prediction'])
for agg, data in agg_dict.items():
    all_agg = pd.concat([all_agg, data[['sub_id', 'prediction']]], axis = 0)
    
all_agg.rename({'sub_id':'id'}, axis = 1, inplace = True)

all_preds = pd.concat([item_predictions[['index', 'prediction']].rename({'index':'id'}, axis = 1), all_agg], axis =0)

for i in np.arange(1,29):
    all_preds[f'F{i}'] = all_preds['prediction']
    
sub = pd.merge(pd.DataFrame(sampsub.id), all_preds.drop('prediction', axis = 1), left_on = 'id', right_on = 'id')
sub.shape

(771120, 29)

In [15]:
sub.to_csv('sub_quantiles.csv', index = False)

In [16]:
sub

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,Total_X_0.005_validation,279.04,279.04,279.04,279.04,279.04,279.04,279.04,279.04,279.04,...,279.04,279.04,279.04,279.04,279.04,279.04,279.04,279.04,279.04,279.04
1,CA_X_0.005_validation,117.92,117.92,117.92,117.92,117.92,117.92,117.92,117.92,117.92,...,117.92,117.92,117.92,117.92,117.92,117.92,117.92,117.92,117.92,117.92
2,TX_X_0.005_validation,77.52,77.52,77.52,77.52,77.52,77.52,77.52,77.52,77.52,...,77.52,77.52,77.52,77.52,77.52,77.52,77.52,77.52,77.52,77.52
3,WI_X_0.005_validation,83.6,83.6,83.6,83.6,83.6,83.6,83.6,83.6,83.6,...,83.6,83.6,83.6,83.6,83.6,83.6,83.6,83.6,83.6,83.6
4,CA_1_X_0.005_validation,36.12,36.12,36.12,36.12,36.12,36.12,36.12,36.12,36.12,...,36.12,36.12,36.12,36.12,36.12,36.12,36.12,36.12,36.12,36.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771115,FOODS_3_823_WI_3_0.995_evaluation,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
771116,FOODS_3_824_WI_3_0.995_evaluation,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
771117,FOODS_3_825_WI_3_0.995_evaluation,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
771118,FOODS_3_826_WI_3_0.995_evaluation,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
